In [45]:
import numpy as np

#load eigenvector information
#for the purpose of easy calculation, here we load structure information

from ase.io import read
atoms=read('data.vasp',format='vasp')

#all atom is in central region by definition
#here we assume number of ghost shell is 1

#construct the list of vector connecting between each atoms

masses=atoms.get_masses()
natom=len(atoms.positions)

In [46]:
with open('gulp.eig','r') as eig:
    lines=eig.readlines()
    #header: number of atom, atom position, number of k-point, number of modes
    ref_natom=int(lines[0])
    if natom!=ref_natom:
        assert "number of atom is different"
    ref_nkpoints=int(lines[natom+1])
    if ref_nkpoints!=0:
        assert "number of kpoint is not zero"
    ref_nmodes=int(lines[natom+2])
    if ref_nmodes!=3*natom:
        assert "number of modes seems strange... format error?"
    
    #lines[natom+3] --> Kpoint coord

    start=natom+4
    mode_block=natom+10
    nmodes=3*natom
    omega=np.zeros(nmodes)
    eigenvector=np.zeros((nmodes,natom,3))

    for imode in range(nmodes):
        omega[imode]=float(lines[start+1])
        eigenv_mode=np.loadtxt(lines[start+2: start+2+natom])
        eigenvector[imode]=eigenv_mode

        start=start+2+natom 

In [47]:
omega

array([-1.40000000e-05, -6.00000000e-06,  1.40000000e-05,  2.21940168e+02,
        2.21940168e+02,  2.21940168e+02,  2.21940168e+02,  2.21940168e+02,
        2.21940168e+02,  4.29619339e+02,  4.29619339e+02,  4.29619339e+02,
        4.29619339e+02,  4.29619339e+02,  4.29619339e+02,  5.16419661e+02,
        5.16419661e+02,  5.16419661e+02,  5.16419661e+02,  5.16419661e+02,
        5.16419661e+02,  5.89804580e+02,  5.89804580e+02,  5.89804580e+02])

In [48]:
eigenvector[0]

array([[-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464]])

In [49]:
eigenvector[1]

array([[0.00939437, 0.0016647 , 0.35342464],
       [0.00939437, 0.0016647 , 0.35342464],
       [0.00939437, 0.0016647 , 0.35342464],
       [0.00939437, 0.0016647 , 0.35342464],
       [0.00939437, 0.0016647 , 0.35342464],
       [0.00939437, 0.0016647 , 0.35342464],
       [0.00939437, 0.0016647 , 0.35342464],
       [0.00939437, 0.0016647 , 0.35342464]])

In [50]:
eigenvector.T[0]

array([[-0.0810933 ,  0.00939437, -0.34399945,  0.06927493, -0.13156827,
        -0.34910638,  0.10340464, -0.1449356 ,  0.27261081,  0.27645266,
        -0.04594068, -0.01720995,  0.15691473, -0.27834608, -0.26280873,
        -0.10807929,  0.31187346, -0.1074178 , -0.05337865,  0.16864587,
        -0.31340791,  0.        ,  0.0060635 , -0.35350139],
       [-0.0810933 ,  0.00939437, -0.34399945, -0.06927493,  0.13156827,
         0.34910638, -0.10340464,  0.1449356 , -0.27261081,  0.27645266,
        -0.04594068, -0.01720995,  0.15691473, -0.27834608, -0.26280873,
         0.10807929, -0.31187346,  0.1074178 ,  0.05337865, -0.16864587,
         0.31340791, -0.        ,  0.0060635 , -0.35350139],
       [-0.0810933 ,  0.00939437, -0.34399945,  0.03593061,  0.06635197,
        -0.15144365, -0.2952307 ,  0.34465896,  0.12417834, -0.27645266,
         0.04594068,  0.01720995, -0.15691473,  0.27834608,  0.26280873,
         0.25401387,  0.15020433,  0.11000857, -0.37702497, -0.08185595,
  

In [51]:
eigen_flat=np.reshape(eigenvector,(nmodes, natom*3))

In [52]:
GULP_eigen=np.loadtxt('eigr.dat').reshape(natom*3,nmodes)

In [53]:
eigen_flat.T[0]

array([-0.0810933 ,  0.00939437, -0.34399945,  0.06927493, -0.13156827,
       -0.34910638,  0.10340464, -0.1449356 ,  0.27261081,  0.27645266,
       -0.04594068, -0.01720995,  0.15691473, -0.27834608, -0.26280873,
       -0.10807929,  0.31187346, -0.1074178 , -0.05337865,  0.16864587,
       -0.31340791,  0.        ,  0.0060635 , -0.35350139])

In [54]:
GULP_eigen[0]

array([-8.10932971e-02,  9.39437410e-03, -3.43999452e-01,  6.92749345e-02,
       -1.31568273e-01, -3.49106384e-01,  1.03404644e-01, -1.44935604e-01,
        2.72610814e-01,  2.76452656e-01, -4.59406784e-02, -1.72099550e-02,
        1.56914734e-01, -2.78346076e-01, -2.62808730e-01, -1.08079287e-01,
        3.11873457e-01, -1.07417803e-01, -5.33786463e-02,  1.68645874e-01,
       -3.13407912e-01,  2.23281552e-15,  6.06350434e-03, -3.53501392e-01])

In [55]:
np.where(np.abs(eigen_flat.T-GULP_eigen) >0.001)

9.304628992608854e-08

In [56]:
masses=atoms.get_masses()
dist=np.zeros((natom,natom,3))
for i in range(natom):
    for j in range(natom):
        rmin=np.zeros(3)
        rij=atoms.get_distance(i,j,vector=True)
        rij_mic=atoms.get_distance(i,j,mic=True,vector=True)
        if(np.linalg.norm(rij_mic)<np.linalg.norm(rij)):
            rmin=rij_mic
        else:
            rmin=rij
        dist[i,j]=rmin

In [57]:
with open('FORCE_CONSTANTS_2ND','r') as fc:
    lines=fc.readlines()

    nlineblock=4
    fc_all=np.zeros((natom,natom,3,3))
    start=1
    for i in range(natom):
        for j in range(natom):
            fc_block=lines[start+1:start+nlineblock]
            fc=np.loadtxt(fc_block)
            fc_all[i,j]=fc/np.sqrt(masses[i]*masses[j])
            #fc_all[i,j]=fc
            start=start+nlineblock

In [58]:
Rx=dist[:,:,0]
Ry=dist[:,:,1]
Rz=dist[:,:,2]

In [59]:
Vx=np.zeros((natom,natom,3,3))
Vy=np.zeros((natom,natom,3,3))
Vz=np.zeros((natom,natom,3,3))

for i in range(natom):
    for j in range(natom):
        Vijx=-Rx[i,j]*fc_all[i,j]
        Vx[i,j]=Vijx

        Vijy=-Ry[i,j]*fc_all[i,j]
        Vy[i,j]=Vijy

        Vijz=-Rz[i,j]*fc_all[i,j]
        Vz[i,j]=Vijz

In [66]:
flatVx=np.reshape(Vx.transpose(0,2,1,3),(24,24))
flatVy=np.reshape(Vy.transpose(0,2,1,3),(24,24))
flatVz=np.reshape(Vz.transpose(0,2,1,3),(24,24))

#check
GULP_Vijz=np.loadtxt('Vijz.dat').reshape((24,24))
np.max(flatVz-GULP_Vijz)

0.21326891228761644

In [62]:
tmp1=np.dot(flatVx,eigen_flat.T)
EVijx=np.dot(eigen_flat,tmp1)

>It seems the result not matches with that of GULP...
--> The reason was the different order of eigenvector in GULP and GULP output

In [63]:
EVijx[0]

array([-8.68350966e-18, -9.38308082e-18,  1.67840440e-18, -2.36636539e-02,
        5.76299967e-03, -3.39307369e-02, -3.50340444e-03, -3.81416093e-03,
       -3.53560806e-02, -4.75536014e-03,  2.07240839e-03,  1.10223893e-06,
       -1.58705960e-03,  1.08718278e-02, -1.37681366e-03,  1.31804443e-02,
       -3.49321524e-02,  1.26966486e-02, -8.24026410e-03,  1.26080404e-02,
       -3.54702031e-02,  5.30290343e-01,  8.23743825e-04,  1.41294230e-05])